# This is Jeopardy!

#### Overview

This project is slightly different than others you have encountered thus far. Instead of a step-by-step tutorial, this project contains a series of open-ended requirements which describe the project you'll be building. There are many possible ways to correctly fulfill all of these requirements, and you should expect to use the internet, Codecademy, and/or other resources when you encounter a problem that you cannot easily solve.

#### Project Goals

You will work to write several functions that investigate a dataset of _Jeopardy!_ questions and answers. Filter the dataset for topics that you're interested in, compute the average difficulty of those questions, and train to become the next Jeopardy champion!

## Prerequisites

In order to complete this project, you should have completed the Pandas lessons in the <a href="https://www.codecademy.com/learn/paths/analyze-data-with-python">Analyze Data with Python Skill Path</a>. You can also find those lessons in the <a href="https://www.codecademy.com/learn/data-processing-pandas">Data Analysis with Pandas course</a> or the <a href="https://www.codecademy.com/learn/paths/data-science/">Data Scientist Career Path</a>.

Finally, the <a href="https://www.codecademy.com/learn/practical-data-cleaning">Practical Data Cleaning</a> course may also be helpful.

## Project Requirements

1. We've provided a csv file containing data about the game show _Jeopardy!_ in a file named `jeopardy.csv`. Load the data into a DataFrame and investigate its contents. Try to print out specific columns.

   Note that in order to make this project as "real-world" as possible, we haven't modified the data at all - we're giving it to you exactly how we found it. As a result, this data isn't as "clean" as the datasets you normally find on Codecademy. More specifically, there's something odd about the column names. After you figure out the problem with the column names, you may want to rename them to make your life easier for the rest of the project.
   
   In order to display the full contents of a column, we've added this line of code for you:
   
   ```py
   pd.set_option('display.max_colwidth', None)
   ```

In [45]:
import pandas as pd
import re
from bs4 import BeautifulSoup, Tag, NavigableString
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
jeopardy = pd.read_csv("jeopardy.csv")

Data cleaning:
* Strip whitespace from beginning/end of column names, remove spaces between column name words, and make column names all lower case
* Make sure that all game rounds are lower case and without punctuation
* Check for null fields in dataframe & remove or repair
* Fix the two answers that are 'null'
* Check the categories column, make it lower case & remove/replace punctuation
* Find duplicated questions and remove duplicates
* Find all video/audio/image questions that don't have an associated file link and remove them
* Find all link based questions and create a new column for the links, then replace the links with link_text
* 

In [47]:
#rename the columns with useable names - remove whitespace, use lower case, change round and value to non-reserved names
jeopardy = jeopardy.rename(columns = {'Show Number':'show_number', ' Air Date':'air_date', ' Round':'game_round', ' Category':'category', ' Value': 'dollar_value',' Question':'question', ' Answer':'answer'})

In [48]:
#Remove punctuation from game rounds
#print(jeopardy.game_round.unique())
jeopardy['game_round'] = jeopardy.game_round.apply(lambda x: x.replace('!', ''))
#print(jeopardy.game_round.unique())

In [79]:
#Categories - remove whitespace, create new column that's just the words from the category (for analysis) without punctuation, but replace '!' with 'exclamation mark [altered]'
#print(jeopardy.category.count())
#print(jeopardy.category.nunique())
jeopardy['category'] = jeopardy.category.apply(lambda x: x.strip())
jeopardy['category_words']= jeopardy['category']
jeopardy['category_words'] = jeopardy.category.apply(lambda x: x.lower())
exclaim_ind = jeopardy.index[jeopardy.category_words =='"!"'].to_list()
for idx in exclaim_ind:
    jeopardy.at[idx, 'category_words'] = "exclamation mark [altered]"

216121


In [50]:
#Function to remove the special characters from a column and just let the words exist in a new column
def replace_in_column (column, **kwargs):
    for key, value in kwargs.items():
        altered_col = column.str.replace(key, value, regex = True)
    return altered_col
replace_punctuation = {'&': 'and','-':'\ ', '_*': '(blank)', '[^\w\s]':''}
jeopardy['category_words'] = replace_in_column(jeopardy['category_words'],**replace_punctuation)

**NOTE**
The below code is a little slow. Consider how to re-write it?

In [52]:
#Function to create a link column from the links in text string in a new column
def link_finder (column):
       soup = BeautifulSoup(column, 'html.parser')
       links = soup.findAll('a', href = True)
       urls = []
       if links != []:
            for link in links:
                     url = link['href']
                     urls.append(url)
       else:
            urls.append("no link")
       urls = ", ".join(urls)
       return urls
jeopardy['link_url'] = jeopardy.question.apply(link_finder)


In [54]:
#Check for duplicate questions & filter to unique questions
#Remove lines containing filler questions,missing audio/video clues, There are several questions that contain [video clue], [audio clue], or [filler] with no link or data on the question, remove those
odd_questions = jeopardy[jeopardy.question.str.contains('\[')]
bad_questions = ['[audio clue]', '[video clue]', '[filler]']
filter_questions = jeopardy[jeopardy.question.isin(bad_questions)]
jeopardy = pd.concat([jeopardy, filter_questions]).drop_duplicates(keep = False)
# Drop any remaining duplicate questions
jeopardy = jeopardy.drop_duplicates('question')
duplicate_questions = jeopardy.question.duplicated().any()
#print(duplicate_questions)

**Slow Code below** Why?

In [55]:
#lambda to create a question column with no links or punctuation
jeopardy['question_no_link'] =jeopardy.question.apply(lambda text: BeautifulSoup(text, 'html.parser').get_text())


In [56]:
jeopardy['question_no_link'] = replace_in_column(jeopardy['question_no_link'], **replace_punctuation)
jeopardy['question_no_link'] = jeopardy.question_no_link.str.lower()

In [57]:
#The answer to two of the questions is 'null' so it got pulled in as a Null value. Let's see if we can fix that.
#Function to replace Null values with the word 'null' when appropriate
def null_value_fixer (df, column_name):
    column = df[column_name]
    indices = (df.index.values[column.isnull()])
    new_df = df.copy()
    for indx in indices:
        new_df.loc[indx, column_name] = '[null]'
    return new_df

jeopardy = null_value_fixer(jeopardy, 'answer')

In [58]:
#Clean answers by duplicating column to one with just the words & text, removing punctuation and making lower case 
jeopardy['answer_words'] = replace_in_column(jeopardy['answer'], **replace_punctuation)
jeopardy['answer_words'] = jeopardy.answer_words.str.lower()

In [59]:
#Remove punctuation from dollar_value
jeopardy['dollar_value'] = jeopardy.dollar_value.str.replace('(\D+)', '', regex= True)

2. Write a function that filters the dataset for questions that contains all of the words in a list of words. For example, when the list `["King", "England"]` was passed to our function, the function returned a DataFrame of 49 rows. Every row had the strings `"King"` and `"England"` somewhere in its `" Question"`.

   Test your function by printing out the column containing the question of each row of the dataset.

****NOTE: AND not OR

In [60]:
#function to find words from a words list in a dataframe column, count them, and return dictionaris of the word:count, and the word:column-rows. 
def words_list_counter (column, words_list):
    wordlist_questions = {}
    word_list_count = {}
    for word in words_list:
        word = word.lower()
        count = column[column.str.contains(word)].count()
        word_questions = column[column.str.contains(word)].reset_index(drop=True)
        word_list_count[word]=count
        wordlist_questions [word] = word_questions
    return word_list_count, wordlist_questions


In [63]:
#test the function to find questions about topics with terrains.
terrain_list = ["land","plain", "sea", "ocean"]
terrain_questions_count, terrain_questions = words_list_counter(jeopardy['question'], terrain_list)
print(terrain_questions_count)
#print(terrain_questions)

{'land': 8520, 'plain': 420, 'sea': 3844, 'ocean': 325}


This breaks on finding islands, and researchers and many countries with names that start with 'land'. It also captures many proper nouns like the name "Garland" and words like 'island'.

3. Test your original function with a few different sets of words to try to find some ways your function breaks. Edit your function so it is more robust.

   For example, think about capitalization. We probably want to find questions that contain the word `"King"` or `"king"`.
   
   You may also want to check to make sure you don't find rows that contain substrings of your given words. For example, our function found a question that didn't contain the word `"king"`, however it did contain the word `"viking"` &mdash; it found the `"king"` inside `"viking"`. Note that this also comes with some drawbacks &mdash; you would no longer find questions that contained words like `"England's"`.

In [64]:
#function to find words from a words list in a dataframe column, count them, and return dictionaries of the word:count, and the word:column-rows. 
def words_list_counter (df, column_name, words_list):
    column = df[column_name]
    wordlist_questions = {}
    word_list_count = {}
    proper_noun_questions = {}
    for word in words_list:
        word = word.lower()
        regex_word = ('\W'+word+'\W')
        word_topic_question = df[column.str.contains(regex_word, regex=True)]
        wordlist_questions [word] = word_topic_question
        word_list_count [word] = word_topic_question[column_name].count()
    return word_list_count, wordlist_questions, 

In [65]:
#test the function to find questions about topics with terrains.
terrain_list = ["land", 'Ocean', 'Sea', 'plain']
terrain_questions_count, terrain_questions= words_list_counter(jeopardy, 'question_no_link', terrain_list)
print(terrain_questions_count)

{'land': 784, 'ocean': 300, 'sea': 988, 'plain': 129}


4. We may want to eventually compute aggregate statistics, like `.mean()` on the `" Value"` column. But right now, the values in that column are strings. Convert the`" Value"` column to floats. If you'd like to, you can create a new column with float values.

   Now that you can filter the dataset of question, use your new column that contains the float values of each question to find the "difficulty" of certain topics. For example, what is the average value of questions that contain the word `"King"`?
   
   Make sure to use the dataset that contains the float values as the dataset you use in your filtering function.

****NOTE: Alter this to drop the null value questions in the averaging, Right answer depends on kind of null value. In this case it's not a zero it's a variable value.

In [67]:
#convert all the final jeopardy questions to a value of 0 so that I can convert the value column to float
final_jeopardy_no_val = jeopardy[jeopardy['dollar_value']=='']
no_val_indx = (jeopardy.index.values[jeopardy['dollar_value']==''])
jeopardy_temp = jeopardy.copy()
for indx in no_val_indx:
    jeopardy_temp.loc[indx, 'dollar_value'] = 0
jeopardy['dollar_value'] = jeopardy_temp['dollar_value'].astype(float)

Average values for questions:

In [69]:
#average value of all questions:
avg_value = (jeopardy['dollar_value'].mean().round(2))
print(avg_value)

740.25


In [19]:
# average value of questions by jeopardy round - Tiebreaker and Final Jeopardy rounds do not have values listed
avg_round_value = lambda x: x[x['game_round'].isin(['Double Jeopardy', 'Jeopardy'])].groupby('game_round').dollar_value.mean().round(2)
print(avg_round_value(jeopardy))

game_round
Double Jeopardy    1017.47
Jeopardy            491.78
Name: dollar_value, dtype: float64


5. Write a function that returns the count of unique answers to all of the questions in a dataset. For example, after filtering the entire dataset to only questions containing the word `"King"`, we could then find all of the unique answers to those questions. The answer "Henry VIII" appeared 55 times and was the most common answer.

In [70]:
def unique_high_freq_answers (df,answers_column, frequency_threshold):
    answers_counts = pd.DataFrame(df[answers_column].value_counts())
    answers_counts.reset_index(inplace=True)
    answers_counts = answers_counts.rename(columns={'index': 'answer', 'answer_words': 'answer_freq'})
    high_freq_ans = answers_counts[answers_counts['answer_freq']>= frequency_threshold]
    return high_freq_ans

In [73]:
jeopardy_unique_answers = (unique_high_freq_answers(jeopardy, 'answer_words', 50))
print(jeopardy_unique_answers)

                       answer  answer_freq
0                   australia          213
1                       china          213
2                       japan          194
3                     chicago          194
4                      france          189
5                       india          185
6                  california          179
7                      canada          173
8                       spain          168
9                      mexico          163
10                     alaska          160
11                      italy          159
12                     hawaii          157
13                      texas          152
14                      paris          148
15                    germany          142
16                     russia          141
17                    florida          139
18               south africa          138
19                    ireland          135
20                     london          133
21                     brazil          133
22         

In [22]:
jeopardy.columns

Index(['show_number', 'air_date', 'game_round', 'category', 'dollar_value',
       'question', 'answer', 'category_words', 'link_url', 'question_no_link',
       'answer_words'],
      dtype='object')

In [74]:
class String_in_column_freq:
    #df = jeopardy.copy().drop('link_url', axis = 1)

    def __init__(self, df, filter_column, target_column, topic_strings, freq_threshold) -> None:
        self.df = df
        self.filter_name = filter_column
        self.target_name = target_column
        #self.column = String_in_column_freq.df[target_column]
        self.column = df[target_column]
        #self.filter = String_in_column_freq.df[filter_column]
        self.filter = df[filter_column]
        self.topics = topic_strings
        self.freq_threshold = freq_threshold

        self.filter_column_inc_topic = {}
        self.filter_column_topic_count = {}
        self.target_topic_above_freq_threshold = {}
    
    def show(self):
        topics_as_string = ", ".join(map(str, self.topics))
        print('Within the jeopardy dataframe we are looking at the frequency with which strings occur in a column')
        print('This class allows us to find all rows of the dataframe where we values frome a list of topics ('+ topics_as_string + ') in the filter column of: ' + self.filter_name)
        print('It then counts the frequency of values in '+ self.target_name+ ' that occur more frequently than the threshold value of ' + str(self.freq_threshold))
        print('The target column is: '+ self.target_name)
        print('The strings are: '+ topics_as_string)
        print('The filter column is: '+ self.filter_name)
        print('The frequency threshold for target column values is: '+ str(self.freq_threshold))
        print('To access the frequency with which our topics show up in the filter column, call topic filter and access self.filter_column topic count')
        print('To access the frequency of values in the target column associated with the filter topics, call high_frequency_targets and access self.target_topic_above_freq_threshold')

    def topic_filter(self):
        for topic in self.topics:
            topic = topic.lower()
            #in case topic is one word that can be part of compound words, this regex will limit it to just the word
            regex_topic = ('\W'+topic+'\W')
            #topic_in_filter = String_in_column_freq.df[self.filter.str.contains(regex_topic, regex=True)]
            topic_in_filter = self.df[self.filter.str.contains(regex_topic, regex=True)]
            self.filter_column_inc_topic [topic] = topic_in_filter
            self.filter_column_topic_count [topic] = topic_in_filter[self.filter_name].count()
        return self.filter_column_inc_topic, self.filter_column_topic_count
    
    def high_frequency_targets (self):
        for key, value in self.filter_column_inc_topic.items():
            target_counts = pd.DataFrame(value[self.target_name].value_counts())
            target_counts.reset_index(inplace=True)
            target_counts = target_counts.rename(columns={'index': 'target', self.target_name: 'target_freq'})
            target_count_above_freq_threshold = target_counts[target_counts['target_freq']>=self.freq_threshold]
            self.target_topic_above_freq_threshold[key] = target_count_above_freq_threshold
    
    def topic_freq_report (self):
        print()

In [76]:
jeopardy_royalty_topics = ['King', 'Queen']
df = jeopardy.copy().drop('link_url', axis = 1)
royalty = String_in_column_freq(df,
                                filter_column='question_no_link',
                                target_column= 'answer_words', 
                                topic_strings=jeopardy_royalty_topics,
                                freq_threshold= 10)
#print(royalty.df.head(10))
#print(royalty.topics)
royalty.topic_filter()
print(royalty.show())
royalty.high_frequency_targets()
print(royalty.target_topic_above_freq_threshold)


Within the jeopardy dataframe we are looking at the frequency with which strings occur in a column
This class allows us to find all rows of the dataframe where we values frome a list of topics (King, Queen) in the filter column of: question_no_link
It then counts the frequency of values in answer_words that occur more frequently than the threshold value of 10
The target column is: answer_words
The strings are: King, Queen
The filter column is: question_no_link
The frequency threshold for target column values is: 10
To access the frequency with which our topics show up in the filter column, call topic filter and access self.filter_column topic count
To access the frequency of values in the target column associated with the filter topics, call high_frequency_targets and access self.target_topic_above_freq_threshold
None
{'king':                      target  target_freq
0                henry viii           35
1               richard iii           23
2                   solomon           

In [77]:
royalty.freq_threshold =5

In [78]:
royalty.topic_filter()
print(royalty.show())
royalty.high_frequency_targets()
print(royalty.target_topic_above_freq_threshold)

Within the jeopardy dataframe we are looking at the frequency with which strings occur in a column
This class allows us to find all rows of the dataframe where we values frome a list of topics (King, Queen) in the filter column of: question_no_link
It then counts the frequency of values in answer_words that occur more frequently than the threshold value of 5
The target column is: answer_words
The strings are: King, Queen
The filter column is: question_no_link
The frequency threshold for target column values is: 5
To access the frequency with which our topics show up in the filter column, call topic filter and access self.filter_column topic count
To access the frequency of values in the target column associated with the filter topics, call high_frequency_targets and access self.target_topic_above_freq_threshold
None
{'king':                      target  target_freq
0                henry viii           35
1               richard iii           23
2                   solomon           21

In [44]:
print((royalty.filter_column_inc_topic.values()))

dict_values([        show_number    air_date       game_round  \
40             4680  2004-12-31  Double Jeopardy   
50             4680  2004-12-31  Double Jeopardy   
545            3036  1997-11-10         Jeopardy   
781            4335  2003-06-06         Jeopardy   
811            4335  2003-06-06  Double Jeopardy   
896            3834  2001-04-12         Jeopardy   
1074           4085  2002-05-10         Jeopardy   
1132           1279  1990-03-08         Jeopardy   
1171           1279  1990-03-08  Double Jeopardy   
1339           3632  2000-05-23  Double Jeopardy   
1345           3632  2000-05-23  Double Jeopardy   
1398           3362  1999-03-30         Jeopardy   
1400           3362  1999-03-30  Double Jeopardy   
1467           4960  2006-03-17  Double Jeopardy   
1675           5580  2008-12-05         Jeopardy   
2277           5630  2009-02-13         Jeopardy   
2307           5630  2009-02-13  Double Jeopardy   
2493           5145  2007-01-12  Double Jeopardy   

6. Explore from here! This is an incredibly rich dataset, and there are so many interesting things to discover. There are a few columns that we haven't even started looking at yet. Here are some ideas on ways to continue working with this data:

 * Investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word `"Computer"` compared to questions from the 2000s?
 * Is there a connection between the round and the category? Are you more likely to find certain categories, like `"Literature"` in Single Jeopardy or Double Jeopardy?
 * Build a system to quiz yourself. Grab random questions, and use the <a href="https://docs.python.org/3/library/functions.html#input">input</a> function to get a response from the user. Check to see if that response was right or wrong.

## Solution

7. Compare your program to our <a href="https://content.codecademy.com/PRO/independent-practice-projects/jeopardy/jeopardy_solution.zip">sample solution code</a> - remember, that your program might look different from ours (and probably will) and that's okay!

8. Great work! Visit <a href="https://discuss.codecademy.com/t/this-is-jeopardy-challenge-project-python-pandas/462365">our forums</a> to compare your project to our sample solution code. You can also learn how to host your own solution on GitHub so you can share it with other learners! Your solution might look different from ours, and that's okay! There are multiple ways to solve these projects, and you'll learn more by seeing others' code.